In [1]:
from torch.utils.data import DataLoader,Dataset
import torchvision.datasets as dset
import numpy as np
from PIL import Image
import random
import torch
from torch import nn
from torch.nn import functional

In [2]:
class  FacesDataset(Dataset):
    
    def __init__(self,imageFolderDataset):
        self.imageFolderDataset = imageFolderDataset
        self.base_url = "./data/orl_faces/"
        
    def __getitem__(self, index):
        anchor_path, anchor_class = random.choice(self.imageFolderDataset.imgs)
        anchor_split_path = anchor_path.split(".")[1].split("/")
        anchor_number = anchor_split_path[-1]
        anchor_path_class = anchor_split_path[-2]
        positive_number = anchor_number
        while positive_number == anchor_number:
            positive_number = np.random.randint(11)
        positive_path = self.base_url + anchor_path_class + "/" + str(positive_number) + ".pgm.png"
        negative_class = anchor_class
        while negative_class == anchor_class:
            negative_class = np.random.randint(40)
        negative_id = np.random.randint(11)
        negative_class_text = ''
        for text_class, int_class in self.imageFolderDataset.class_to_idx.items():
            if int_class == negative_class:
                negative_class_text = text_class
        negative_path = self.base_url + negative_class_text + "/" + str(negative_id) + ".pgm.png"
        
        anchor_image = Image.open(anchor_path)
        anchor_image = torch.FloatTensor(np.array(anchor_image.convert("L"))).unsqueeze(0)
        positive_image = Image.open(positive_path)
        positive_image = torch.FloatTensor(np.array(positive_image.convert("L"))).unsqueeze(0)
        negative_image = Image.open(negative_path)
        negative_image = torch.FloatTensor(np.array(negative_image.convert("L"))).unsqueeze(0)
        
        return anchor_image, positive_image, negative_image
    
    def __len__(self):
        return len(self.imageFolderDataset.imgs)

In [3]:
folder_dataset = dset.ImageFolder("./data/orl_faces/")
fdata = FacesDataset(folder_dataset)
fdataloader  = DataLoader(fdata,
                        shuffle=True,
                        num_workers=1,
                        batch_size=1)

In [84]:
class SiameseNet(nn.Module):
    def __init__(self):
        super(SiameseNet, self).__init__()
        self.cnn1 = nn.Conv2d(in_channels=1, out_channels=64, kernel_size=5, stride=3)
        self.cnn2 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=5, stride=3)
        self.bn1 = nn.BatchNorm2d(32)
        self.fc1 = nn.Linear(11*9*32, 128)
        
    def forward_once(self, x):
        x = self.cnn1(x)
        x = self.cnn2(x)
        x = functional.relu(x)
        x = self.bn1(x)
        x = functional.dropout2d(x)
        x = self.fc1(x.view(-1, 11*9*32))
        return x
        
    def forward(self, anchor, positive, negative):
        anchor_embedding = self.forward_once(x)
        positive_embedding = self.forward_once(positive)
        negative_embedding = self.forward_once(negative)
        positive_distance = functional.pairwise_distance(anchor_embedding, positive_embedding)
        negative_distance = functional.pairwise_distance(anchor_embedding, negative_embedding)
        return positive_distance

In [85]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = SiameseNet().to(device)

In [86]:
for i, (anchor, positive, negative) in enumerate(fdataloader):
    model.forward_once(anchor)

torch.Size([1, 32, 11, 9])
torch.Size([1, 128])


FileNotFoundError: Traceback (most recent call last):
  File "/Users/tfolkman/anaconda3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 57, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/Users/tfolkman/anaconda3/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 57, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-58-a9590caf23f0>", line 30, in __getitem__
    negative_image = Image.open(negative_path)
  File "/Users/tfolkman/anaconda3/lib/python3.5/site-packages/PIL/Image.py", line 2477, in open
    fp = builtins.open(filename, "rb")
FileNotFoundError: [Errno 2] No such file or directory: './data/orl_faces/s21/0.pgm.png'
